# Eval Starter Kit Notebook

In [1]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

print(f"This is the repo dir {repo_dir}")

This is the repo dir /Users/kwasia/Documents/Projects/ai-starter-kit


In [2]:
import pandas as pd
from utils.eval.rag_eval import (
    RAGEvaluator,
    RAGEvalConfig,
    load_pipeline,
    load_eval_dataframe,
    evaluate,
)
from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

/Users/kwasia/.pyenv/versions/3.11.3/envs/rag-eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Use Case 1: CSV file with pre-generated answers, no context
# Load config
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

conf: ('EVAL_LLAMA38B', {'name': 'EVAL_LLAMA38B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA38B
llm_config: {'sambastudio_base_url': 'https://sjc3-demo2.sambanova.net', 'sambastudio_project_id': 'bdadb40a-99d2-4705-96ff-48919435a0d8', 'sambastudio_endpoint_id': 'e0c07d60-3d7b-45f8-8653-088d4b8e8abd', 'sambastudio_api_key': 'd0838e80-e2c5-49f3-8c16-4127f97e9854', 'streaming': True, 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}}
conf: ('EVAL_LLAMA370B', {'name': 'EVAL_LLAMA370B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-70B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA370B
llm_config: {'sambastudio_base_url': 'https://sjc3-demo2.sambanova.net', 'sambastudio_project_id': 'bdadb40a-99d2-4705-96ff-48919435a0d8', 'sambastudio_endpoint_id': 'e0c07d60-3d7b-45f8-8653-088d4b8e8a

2024-07-05 12:50:00 - Load pretrained SentenceTransformer: intfloat/e5-large


load INSTRUCTOR_Transformer


2024-07-05 12:50:03 - Use pytorch device: cpu


max_seq_length  512


In [4]:
eval_df = pd.read_csv("../data/tepsys_eng_tran.csv")
results1 = evaluator.evaluate(eval_df)
print("Results 1:", results1)

2024-07-05 12:50:39 - Evaluating metrics for EVAL_LLAMA38B
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/kwasia/.pyenv/versions/3.11.3/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Users/kwasia/.pyenv/versions/3.11.3/envs/rag-eval/lib/python3.11/site-packages/ragas/executor.py", line 87, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kwasia/.pyenv/versions/3.11.3/lib/python3.11/asyncio/base_events.py", line 629, in run_until_complete
    self._check_running()
  File "/Users/kwasia/.pyenv/versions/3.11.3/lib/python3.11/asyncio/base_events.py", line 588, in _check_running
    raise RuntimeError('This event loop is already running')
RuntimeError: This event loop is already running
/Users/kwasia/.pyenv/versions/3.11.3/lib/python3.11/threading.py:1040: RuntimeWarning: coroutine 'Runner._aresults' was never awaited
  self._i

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

/Users/kwasia/.pyenv/versions/3.11.3/envs/rag-eval/lib/python3.11/site-packages/ipykernel/iostream.py:123: RuntimeWarning: coroutine 'as_completed.<locals>.sema_coro' was never awaited
  await self._event_pipe_gc()


In [ ]:
# Use Case 2: CSV file with pre-generated answers and context, add context_col to eval_dataset yaml section
# Re- Load config after changes
eval_df = pd.read_csv("../data/res.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)


results2 = evaluator.evaluate(eval_df)
print("Results 2:", results2)

In [3]:
# Use Case 3: CSV file without answers, generate with pipelines, no context
# Re- Load config after changes
eval_df = pd.read_csv("../data/tepsys_eng_tran.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [
        config.get_llm_config(conf) for conf in config.llm_configs
    ]
]
results3 = evaluator.evaluate(eval_df, pipelines)
print("Results 3:", results3)

conf: ('EVAL_LLAMA38B', {'name': 'EVAL_LLAMA38B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA38B
llm_config: {'sambastudio_base_url': 'https://sjc3-demo2.sambanova.net', 'sambastudio_project_id': 'bdadb40a-99d2-4705-96ff-48919435a0d8', 'sambastudio_endpoint_id': 'e0c07d60-3d7b-45f8-8653-088d4b8e8abd', 'sambastudio_api_key': 'd0838e80-e2c5-49f3-8c16-4127f97e9854', 'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}}
conf: ('EVAL_LLAMA370B', {'name': 'EVAL_LLAMA370B', 'model_kwargs': {'select_expert': 'Meta-Llama-3-70B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}})
llm_name: EVAL_LLAMA370B
llm_config: {'sambastudio_base_url': 'https://sjc3-demo2.sambanova.net', 'sambastudio_project_id': 'bdadb40a-99d2-4705-96ff-48919435a0d8', 'sambastudio_endpoint_id': 'e0c07d60-3d7b-45f8-8653-088d4b8e8abd', 'sambastudio_a

/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer
max_seq_length  512


/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The Computer Management ledger is a critical tool for tracking and managing computer assets within an organization. To update the Computer Management ledger, you'll want to ensure that you have the most recent and accurate information. Here's a step-by-step guide to help you update the ledger:

**Gather necessary information:**

1. **Hardware and software inventory:** Collect data on all computers, including their make, model, serial number, processor, RAM, storage, operating system, and installed software.
2. **Asset tags and IDs:** Verify the asset tags and IDs assigned to each computer to ensure accurate tracking.
3. **Location and department:** Update the location and department information for each computer to reflect any changes.
4. **User assignments:** Verify the users assigned to each computer and update the ledger accordingly.
5. **Warranty and maintenance information:** Check the warranty status and maintenance records for each computer.

**Update the ledger:**

1. **Review

...

[{'question': 'I want to update the computer management ledger', 'answer': ' To update the computer management ledger, you can follow these steps:\n\n1. Open the Computer Management console: You can do this by searching for "Computer Management" in the Start menu, or by typing "compmgmt.msc" in the Run dialog box (Windows key + R).\n2. Navigate to the System Tools folder: In the Computer Management console, navigate to the System Tools folder.\n3. Click on the System Information tool: In the System Tools folder, click on the System Information tool.\n4. Click on the "System Summary" tab: In the System Information tool, click on the "System Summary" tab.\n5. Update the system information: You can update the system information by clicking on the "Update" button in the System Information tool.\n6. Save the changes: Once you have updated the system information, you can save the changes by clicking on the "Save" button.\n\nAlternatively, you can also update the computer management ledger by

Evaluating: 100%|██████████| 9/9 [02:00<00:00, 13.34s/it]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: kwasi-ankomah (ai-solutions-team). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current proces

Evaluation Table:
                                             question  \
0     I want to update the computer management ledger   
1     I want to update the computer management ledger   
2     I want to update the computer management ledger   
3     I want to update the computer management ledger   
4     I want to update the computer management ledger   
5     I want to update the computer management ledger   
6     I want to update the computer management ledger   
7     I want to update the computer management ledger   
8     I want to update the computer management ledger   
9     I want to update the computer management ledger   
10    I want to update the computer management ledger   
11    I want to update the computer management ledger   
12    I want to update the computer management ledger   
13    I want to update the computer management ledger   
14    I want to update the computer management ledger   
15    I want to update the computer management ledger   
16    I want 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Results 3: {'LLAMA70B_GEN_EVAL_LLAMA38B':                                             question  \
0    I want to update the computer management ledger   
1    I want to update the computer management ledger   
2  Please tell me how to install commercially ava...   

                                              answer  \
0   The Computer Management ledger is a critical ...   
1   The Computer Management ledger is a critical ...   
2   Installing commercially available software on...   

                                        ground_truth contexts  \
0  The PC management ledger will be updated from ...       []   
1  The PC management ledger will be updated from ...       []   
2  If you wish to install non-standard software o...       []   

   answer_relevancy  answer_correctness  answer_similarity  
0          0.956728            0.555109           0.887104  
1          0.956728            0.555109           0.887104  
2          0.966814            0.278824           0.865295  , 'L

In [ ]:
# Use Case 4: CSV file without answers, generate with pipelines, with context from vector DB
# Re- Load config after changes
eval_df = pd.read_csv("../data/res.csv")
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    llm_name, llm_config = config.get_llm_config(conf)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))


eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

eval_df = pd.read_csv("../data/res.csv")
pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [
        config.get_llm_config(conf) for conf in config.llm_configs
    ]
]
results4 = evaluator.evaluate(eval_df, pipelines)
print("Results 4:", results4)

In [ ]:
# Running the evaluation using CLI
config_path = "path/to/config.yaml"
eval_csv_path = "../data/res.csv"
config_path = "../../utils/eval/config.yaml"

# Command without generation
command_no_gen = [
    "python",
    "evaluate.py",
    "--config",
    config_path,
    "--eval_csv",
    eval_csv_path,
]

# Command with generation
command_with_gen = [
    "python",
    "evaluate.py",
    "--config",
    config_path,
    "--eval_csv",
    eval_csv_path,
    "--generation",
]

# Execute command without generation
result_no_gen_cli = subprocess.run(command_no_gen, capture_output=True, text=True)
print("CLI results without generation:")
print(result_no_gen_cli.stdout)

# Execute command with generation
result_with_gen_cli = subprocess.run(command_with_gen, capture_output=True, text=True)
print("CLI results with generation:")
print(result_with_gen_cli.stdout)